In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv("zbiór_10.csv")

In [3]:
X = data.drop(columns=["default"])
y = data["default"]

# dropujemy szczegolna forma wlasnosci (kazdy ma taka sama)
X = X.drop(columns="szczegolnaFormaPrawna_Symbol")

unique_values = X['formaWlasnosci_Symbol'].unique()

categorical_cols = ['formaWlasnosci_Symbol']

# lista kolumn OHE odpowiadających symbolom form własności
numeric_cols = [
    'ohe_fw_214','ohe_fw_215','ohe_fw_113','ohe_fw_216','ohe_fw_225','ohe_fw_226',
    'ohe_fw_224','ohe_fw_227','ohe_fw_234','ohe_fw_111','ohe_fw_112','ohe_fw_235',
    'ohe_fw_132','ohe_fw_123','ohe_fw_133','ohe_fw_122','ohe_fw_338', 'ohe_fw_000'
]

# inicjalizacja OneHotEncoder z ustalonymi kategoriami
ohe = OneHotEncoder(
    categories=[sorted([int(c.split('_')[-1]) for c in numeric_cols])],
    sparse_output=False,  # zmiana z sparse -> sparse_output
    drop=None
)

# dopasowanie i transformacja
ohe_array = ohe.fit_transform(X[['formaWlasnosci_Symbol']])

# utworzenie DataFrame z odpowiednimi nazwami kolumn
df_ohe = pd.DataFrame(ohe_array, columns=numeric_cols, index=X.index)

# połączenie z oryginalnym df
df = pd.concat([X, df_ohe], axis=1)
df = df.drop(columns=["formaWlasnosci_Symbol"])

In [4]:
## analogicznie dla kolumny 'schemat_wsk_bilans'

ohe_cols = ['SFJIN_wsk_bilans', 'SFJMI_wsk_bilans', 'SFJMA_wsk_bilans']

# inicjalizacja OneHotEncoder z ustalonymi kategoriami
ohe = OneHotEncoder(
    categories = [['SFJIN', 'SFJMI', 'SFJMA']],
    sparse_output=False,  # zmiana z sparse -> sparse_output
    drop=None
)

# dopasowanie i transformacja
ohe_array = ohe.fit_transform(X[['schemat_wsk_bilans']])

# utworzenie DataFrame z odpowiednimi nazwami kolumn
df_ohe = pd.DataFrame(ohe_array, columns=ohe_cols, index=X.index)

# połączenie z oryginalnym df
df = pd.concat([df, df_ohe], axis=1)
df = df.drop(columns=["schemat_wsk_bilans"])

In [5]:
## i jeszcze raz dla 'schemat_wsk_rzis'

ohe_cols = ['SFJIN_wsk_rzis', 'SFJMI_wsk_rzis', 'SFJMA_wsk_rzis']

# inicjalizacja OneHotEncoder z ustalonymi kategoriami
ohe = OneHotEncoder(
    categories = [['SFJIN', 'SFJMI', 'SFJMA']],
    sparse_output=False,  # zmiana z sparse -> sparse_output
    drop=None
)

# dopasowanie i transformacja
ohe_array = ohe.fit_transform(X[['schemat_wsk_rzis']])

# utworzenie DataFrame z odpowiednimi nazwami kolumn
df_ohe = pd.DataFrame(ohe_array, columns=ohe_cols, index=X.index)

# połączenie z oryginalnym df
df = pd.concat([df, df_ohe], axis=1)
df = df.drop(columns=["schemat_wsk_rzis"])

In [6]:
# wybieramy top_n najczęstszych kategorii
X = df.copy()
top_n = 10
top_values = X['pkdKod'].value_counts().nlargest(top_n).index

# tworzymy nową kolumnę, gdzie rzadkie wartości są zastąpione przez '0'
X['pkdKod_grouped'] = X['pkdKod'].where(X['pkdKod'].isin(top_values), other='0')

X = X.drop(columns=['pkdKod'])

In [7]:
# łączymy X i y do obliczenia statystyk
df_temp = X.copy()
df_temp['default'] = y.values

# agregujemy liczbe przypadkow
agg = df_temp.groupby('pkdKod_grouped')['default'].agg(['sum', 'count']).rename(columns={'sum': 'bad', 'count': 'total'})
agg['good'] = agg['total'] - agg['bad']

# smoothing, żeby uniknąć dzielenia przez zero
smoothing = 0.5
agg['bad_s'] = agg['bad'] + smoothing
agg['good_s'] = agg['good'] + smoothing

total_bad = agg['bad_s'].sum()
total_good = agg['good_s'].sum()

agg['woe'] = np.log((agg['good_s'] / total_good) / (agg['bad_s'] / total_bad))

woe_map = agg['woe'].to_dict()

# dodajemy nową kolumnę do X
X['WoE_pkdKod_grouped'] = X['pkdKod_grouped'].map(woe_map)

X = X.drop(columns=['pkdKod_grouped'])

In [8]:
# Wywalamy za bardzo skorelowane
corr_matrix = X.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

X.drop(to_drop, axis=1, inplace=True)

In [ ]:
# inf na nan
X = X.replace([np.inf, -np.inf], np.nan)

# kolumny indykacji ze w kolumnie danego wiersza byly braki danych
for c in X.columns:
    if X[c].isna().any():
        X[f"{c}_mial_braki_danych"] = X[c].isna().astype(int)

# najbardziej bezmozgi imputter, nie mam sily poddaje sie
imputer = SimpleImputer(strategy="median")
X[X.columns] = imputer.fit_transform(X)